In [28]:
import sys
sys.path.append("/home/jupyter/.local/bin")

In [2]:
%pip uninstall ts_rnn

Found existing installation: ts-rnn 0.2.1
Uninstalling ts-rnn-0.2.1:
  Successfully uninstalled ts-rnn-0.2.1


In [3]:
%pip install https://github.com/LevPerla/Time_Series_Prediction_RNN/archive/master.zip

Defaulting to user installation because normal site-packages is not writeable
     / 225 kB 1.1 MB/s
  Created wheel for ts-rnn: filename=ts_rnn-0.2.1-py3-none-any.whl size=12122 sha256=1a98750fc324b243b0ba92d22f0dca771e512c873b9765d19d22f2f1133dcca3
  Stored in directory: /tmp/pip-ephem-wheel-cache-vonwvgve/wheels/69/af/0a/eea6ef350c6ad2c73e2b6e6cb010919adc7f9cd34b4e4608a2
Successfully built ts-rnn
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [356]:
# yfinance, keras_tuner, sktime
%pip install yfinance
%pip install sktime

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [454]:
#################################           Load libs                      #############################################
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import yfinance as yf
import json
from copy import deepcopy
from tensorflow.keras import callbacks
from ts_rnn.model import TS_RNN
from keras_tuner import HyperParameters
from ts_rnn.utils import metrics_eval, train_test_pred_plot, train_val_test_pred_plot
from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.get_logger().setLevel('CRITICAL')

from sktime.transformations.series.detrend import Deseasonalizer, ConditionalDeseasonalizer
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.outlier_detection import HampelFilter
from sktime.transformations.series.boxcox import BoxCoxTransformer, LogTransformer

Execute error: Servant c1.4 not allocated: Internal Error

## Config experiment

In [435]:
#!c1.8
#Config
hp = HyperParameters()
rnn_arch = {"layers": [
                        ["LSTM", {"units": hp.Int(name='units',
                                                 min_value=2,
                                                 max_value=30,
                                                 step=10,
                                                 default=12
                                                ),
                                  "return_sequences": False,
                                  "kernel_initializer": "glorot_uniform",
                                  "activation": hp.Choice(name='LSTM_1_activation',
                                                          values=['relu', 'tanh', 'sigmoid', "linear"],
                                                          default='relu'),
                                  }],
                        ["Dropout", {"rate": hp.Float(name='dropout',
                                                      min_value=0.0,
                                                      max_value=0.5,
                                                      default=0.2,
                                                      step=0.05)
                                     }],
                        ["Dense", {"activation": "linear"}]
                    ]}

my_callbacks = [callbacks.EarlyStopping(patience=10, monitor='val_loss')]

CONFIG = {
    "TARGET": {'TICKERS': [
                           'YNDX.ME',
                           'SBER.ME',
                           'POLY.ME',
                           'SIBN.ME',
                           'AMZN',
                           'AAPL',
                           'GOOGL',
                           'NFLX'
                           ],
               'MIN_DATE': '2012-01-01',
               'MAX_DATE': '2022-01-01'},
    "VAL_LEN": 7,
    "TEST_LEN": 7,
    "CV_FOLDS": 5,
    "MODEL": {'INIT':{
                    'rnn_arch': rnn_arch,
                    'tuner_hp': hp,
                    "strategy": "MiMo", # "Direct", "Recursive", "MiMo"
                    "n_lags": 30,
                    "horizon": 7,
                    "tuner": "BayesianOptimization", # "RandomSearch", "BayesianOptimization", "Hyperband"
                    "max_trials": 5,
                    "loss": 'mae',
                    "optimizer": 'adam'
                    },
                'FIT':{"epochs": 40,
                      "batch_size": 14,
                       'callbacks': my_callbacks}
              },
    "OUTLAYER_TRANSFORMERS": ['HampelFilter',
                              None
                              ],
    "BASE_TRANSFORMERS": [
                        'Differencer',
                        'LogTransformer',
                        'BoxCoxTransformer',
                        None
                       ],
    "SEASON_TRANSFORMERS": ['Deseasonalizer',
                            None
                            ],
    "NORM_TRANSPORMERS": ["MinMaxScaler",
                          None
                          ],
    "TRANSFORMERS_ARGS": { 'HampelFilter':{'window_length': 10},
                                'MinMaxScaler': {"feature_range": (0, 1)},
                                'Differencer': {'lags': [1], "drop_na": False},
                                'LogTransformer':{},
                                'BoxCoxTransformer': {},
                                'Deseasonalizer':{'sp': 364,
                                                  'model': 'multiplicative'},
                                }
}

tscv = TimeSeriesSplit(gap=0, n_splits=CONFIG["CV_FOLDS"], test_size=CONFIG["VAL_LEN"]+CONFIG["TEST_LEN"])
n_fits = CONFIG['MODEL']['INIT']['max_trials'] * len(CONFIG['BASE_TRANSFORMERS']) * len(CONFIG['SEASON_TRANSFORMERS']) * CONFIG['CV_FOLDS'] * len(CONFIG['TARGET']['TICKERS'])
f"There will be {n_fits} fits"

'There will be 1600 fits'

In [365]:
#!c1.8
print((4 * 2 *2 *2* 5 * 8) * 4.2 / 60 / 24, 'суток')

3.733333333333333 суток


In [364]:
def seve_fig_from_array(array, path, fig_name):
    plt.plot(array)
    plt.savefig(os.path.join(path, fig_name))
    plt.close()

In [442]:
models_dict = {}
for file_name in os.listdir(exp_folder):
    if '.' not in file_name:
        models_dict[file_name] = {}
        for file_name2 in os.listdir(os.path.join(exp_folder, file_name)):
            models_dict[file_name]['mean_val_metrics'] = {} if 'mean_val_metrics' not in  models_dict[file_name].keys() else models_dict[file_name]['mean_val_metrics']
            models_dict[file_name]['mean_val_metrics'][file_name2] = {}
            models_dict[file_name]['mean_test_metrics'] = {} if 'mean_test_metrics' not in  models_dict[file_name].keys() else models_dict[file_name]['mean_test_metrics']
            models_dict[file_name]['mean_test_metrics'][file_name2] = {}
            
            
            mean_val_df = pd.DataFrame()
            mean_test_df = pd.DataFrame()
            for file_name3 in os.listdir(os.path.join(exp_folder, file_name, file_name2)):
                val_tmp = pd.read_csv(os.path.join(exp_folder, file_name,file_name2, file_name3, 'val_mertics.txt'), sep=':',index_col=0, names=[file_name3])
                mean_val_df = pd.concat([mean_val_df, val_tmp], axis=1)
                
                test_tmp = pd.read_csv(os.path.join(exp_folder, file_name,file_name2, file_name3, 'test_mertics.txt'), sep=':',index_col=0, names=[file_name3])
                mean_test_df = pd.concat([mean_test_df, test_tmp], axis=1)
            
            mean_val_df = mean_val_df.mean(axis=1)
            models_dict[file_name]['mean_val_metrics'][file_name2] = mean_val_df.to_dict()
            mean_test_df = mean_test_df.mean(axis=1)
            models_dict[file_name]['mean_test_metrics'][file_name2] = mean_test_df.to_dict()

In [425]:
models_dict

{'HampelFilter_Deseasonalizer_Differencer_MinMaxScaler': {'mean_val_metrics': {'YNDX': {'Mean Absolute Error': 137.2234,
    'Mean Squared Error': 29185.0452,
    'Symmetric Mean absolute percentage error': 2.5143999999999997,
    'Root Mean Squared Error': 163.28140000000002,
    'Mean absolute percentage error': 2.5438},
   'SBER': {'Mean Absolute Error': 5.8634,
    'Mean Squared Error': 59.436,
    'Symmetric Mean absolute percentage error': 1.8181999999999998,
    'Root Mean Squared Error': 6.7776,
    'Mean absolute percentage error': 1.8381999999999998},
   'POLY': {'Mean Absolute Error': 35.365399999999994,
    'Mean Squared Error': 2822.2388,
    'Symmetric Mean absolute percentage error': 2.6328,
    'Root Mean Squared Error': 43.744600000000005,
    'Mean absolute percentage error': 2.5984},
   'SIBN': {'Mean Absolute Error': 7.0224,
    'Mean Squared Error': 82.20840000000001,
    'Symmetric Mean absolute percentage error': 1.4598,
    'Root Mean Squared Error': 7.990800000

In [378]:
exp_folder = './reports/data_tr_experiment_5'

In [432]:
'HampelFilter_None_Differencer_None' in os.listdir(exp_folder)

False

In [437]:
CONFIG['SEASON_TRANSFORMERS']

['Deseasonalizer', None]

In [443]:
#!c1.4
# Import targets
targets_df = yf.download(CONFIG['TARGET']['TICKERS'],
                         start=CONFIG['TARGET']['MIN_DATE'],
                         end=CONFIG['TARGET']['MAX_DATE']
                        )['Adj Close']
if isinstance(targets_df, pd.Series):
    targets_df.name = CONFIG['TARGET']['TICKERS'][0]
    targets_df = targets_df.to_frame()
full_ind = pd.date_range(targets_df.index.min(), targets_df.index.max())
targets_df = targets_df.reindex(full_ind, fill_value=np.nan).interpolate()

# # Make new experiment folder
# if "reports" not in os.listdir('.'):
#     os.makedirs("./reports")
# new_folder_num = str(len(os.listdir("./reports")) + 1)
# exp_folder = os.path.join("./reports", "data_tr_experiment_" + new_folder_num)
# os.mkdir(exp_folder)
# print(f"Save experiment in {exp_folder}")

# # Save config
# CONFIG_ = deepcopy(CONFIG)
# del CONFIG_['MODEL']['INIT']['tuner_hp']
# del CONFIG_['MODEL']['FIT']['callbacks']
# with open(os.path.join(exp_folder, "exp_config.json"), "w") as outfile:
#     json.dump(CONFIG_, outfile, skipkeys=True)

# models_dict = {}

# Train RNN models
for outlayer_transformer_name in CONFIG['OUTLAYER_TRANSFORMERS']:
    for norm_transformer_name in CONFIG['NORM_TRANSPORMERS']:
        for base_transformer_name in CONFIG['BASE_TRANSFORMERS']:
            for seasonal_transformer_name in CONFIG['SEASON_TRANSFORMERS']:
                model_name =  f"{outlayer_transformer_name}_{seasonal_transformer_name}_{base_transformer_name}_{norm_transformer_name}"
                
                print(seasonal_transformer_name)
                if model_name in os.listdir(exp_folder):
                    continue 
                    
                print(f"Model {model_name}")
                model_folder = os.path.join(exp_folder, model_name)
                os.mkdir(model_folder)

                for ticker_name in CONFIG['TARGET']['TICKERS']:
                    target = targets_df[ticker_name].dropna()
                    series_folder = os.path.join(model_folder, ticker_name.split('.')[0])
                    os.mkdir(series_folder)

                    cv_val_metrics, cv_test_metrics = [], []
                    cv_val_predictions, cv_test_predictions = [], []

                    for train_index, val_index in tscv.split(target):
                        # divide val to val and test
                        test_index = val_index[CONFIG["VAL_LEN"]:]
                        val_index = val_index[:CONFIG["VAL_LEN"]]

                        iter_folder = f"TRAIN_{len(train_index)}_VAL_{len(val_index)}_TEST_{len(test_index)}"
                        print(iter_folder)

                        # Create folder for cv iteration
                        iter_folder = os.path.join(series_folder, iter_folder)
                        plots_folder = os.path.join(iter_folder, 'plots')
                        os.mkdir(iter_folder)
                        os.mkdir(plots_folder)
                        plot_id = 0

                        target_train, target_val, target_test = target[train_index], target[val_index], target[test_index]
                        seve_fig_from_array(target_train, plots_folder, f'{plot_id}_target')
                        plot_id+=1

                        # Transform
                        # Del outlayers
                        if outlayer_transformer_name is not None:
                            outlayer_transformer = eval(outlayer_transformer_name)(**CONFIG['TRANSFORMERS_ARGS'][outlayer_transformer_name])
                            target_train_tr = outlayer_transformer.fit_transform(target_train).interpolate(limit_direction="both")
                            target_val_tr = target_val
                            seve_fig_from_array(target_train_tr, plots_folder, f'{plot_id}_target_outlayers_removed')
                            plot_id+=1
                        else:
                            target_train_tr = target_train
                            target_val_tr = target_val

                        if seasonal_transformer_name is not None:
                            seasonal_transformer = eval(seasonal_transformer_name)(**CONFIG['TRANSFORMERS_ARGS'][seasonal_transformer_name])
                            target_train_tr = seasonal_transformer.fit_transform(target_train_tr)
                            target_val_tr = seasonal_transformer.transform(target_val_tr)
                            seve_fig_from_array(target_train_tr, plots_folder, f'{plot_id}_target_{seasonal_transformer_name}')
                            plot_id+=1

                        if base_transformer_name is not None:
                            base_transformer = eval(base_transformer_name)(**CONFIG['TRANSFORMERS_ARGS'][base_transformer_name])
                            target_train_tr = base_transformer.fit_transform(target_train_tr).interpolate(limit_direction="both")
                            target_val_tr = base_transformer.transform(target_val_tr).interpolate(limit_direction="both")
                            seve_fig_from_array(target_train_tr, plots_folder, f'{plot_id}_target_{base_transformer_name}')
                            plot_id+=1

                        # Normalize target
                        if norm_transformer_name is not None:
                            target_scaler = eval(norm_transformer_name)(**CONFIG['TRANSFORMERS_ARGS'][norm_transformer_name])
                            target_train_tr = target_scaler.fit_transform(target_train_tr.values.reshape(-1, 1))
                            target_val_tr = target_scaler.transform(target_val_tr.values.reshape(-1, 1))
                            seve_fig_from_array(pd.Series(target_train_tr.flatten(), index=target_train.index), plots_folder, f'{plot_id}_target_scaled')
                            plot_id+=1
                        else:
                            target_train_tr = target_train_tr.values.reshape(-1, 1)
                            target_val_tr = target_val_tr.values.reshape(-1, 1)

                        model = TS_RNN(save_dir=iter_folder, **CONFIG["MODEL"]['INIT'])
                        model.logger.info(f'[Experiment] {model_name}')
                        model.logger.info(f'[Experiment] {ticker_name}_TRAIN_{len(train_index)}_VAL_{len(val_index)}_TEST_{len(test_index)}')

                        model.fit(target_train=target_train_tr, target_val=target_val_tr,
                                  **CONFIG["MODEL"]['FIT'],
                                  verbose=1)

                        predicted_val = model.predict(target=target_train_tr[-model.n_lags:],
                                                          prediction_len=CONFIG["VAL_LEN"])
                        predicted_test = model.predict(target=np.concatenate([target_train_tr, target_val_tr])[-model.n_lags:],
                                                          prediction_len=CONFIG["VAL_LEN"])

                        # inverse transform
                        if norm_transformer_name is not None:
                            predicted_val = pd.Series(target_scaler.inverse_transform(predicted_val.reshape(-1, 1))
                                                                      .flatten(),
                                                         index=target_val.index)
                            predicted_test = pd.Series(target_scaler.inverse_transform(predicted_test.reshape(-1, 1))
                                                                      .flatten(),
                                                         index=target_test.index)
                        else:
                            predicted_val = pd.Series(predicted_val, index=target_val.index)
                            predicted_test = pd.Series(predicted_test, index=target_test.index)

                        if base_transformer_name is not None:
                            predicted_val = base_transformer.inverse_transform(predicted_val)
                            if base_transformer_name == 'Differencer':
                                predicted_test = target_val[-1] + predicted_test.cumsum()
                            else:
                                predicted_test = base_transformer.inverse_transform(predicted_test)

                        if seasonal_transformer_name is not None:
                            predicted_val = seasonal_transformer.inverse_transform(predicted_val)
                            predicted_test = seasonal_transformer.inverse_transform(predicted_test)

                        # Calculate metrics
                        val_mertics = metrics_eval(target_val[:len(predicted_val)], predicted_val, save_dir=iter_folder, print_result=False, name="val_mertics")
                        test_mertics = metrics_eval(target_test[:len(predicted_val)], predicted_test, save_dir=iter_folder, print_result=False, name='test_mertics')

                        cv_val_metrics.append(val_mertics)
                        cv_test_metrics.append(test_mertics)

                        cv_val_predictions.append(predicted_val)
                        cv_test_predictions.append(predicted_test)

                        train_val_test_pred_plot(train=target_train,
                                                 val=target_val,
                                                 test=target_test,
                                                 val_pred=predicted_val,
                                                 test_pred=predicted_test,
                                                 save_dir=plots_folder, show=False)

                    models_dict[model_name] = {} if model_name not in models_dict else models_dict[model_name]
                    models_dict[model_name]["mean_val_metrics"] = {} if "mean_val_metrics" not in models_dict[model_name] else models_dict[model_name]["mean_val_metrics"]
                    models_dict[model_name]["mean_test_metrics"] = {} if "mean_test_metrics" not in models_dict[model_name] else models_dict[model_name]["mean_test_metrics"]
                    models_dict[model_name]["mean_val_metrics"][ticker_name] = dict(pd.DataFrame(cv_val_metrics).mean())
                    models_dict[model_name]["mean_test_metrics"][ticker_name] = dict(pd.DataFrame(cv_test_metrics).mean())

Trial 5 Complete [00h 01m 02s]
val_loss: 0.081609345972538

Best val_loss So Far: 0.081609345972538
Total elapsed time: 00h 04m 54s
INFO:tensorflow:Oracle triggered exit
Deseasonalizer
None


In [445]:
%pip install pmdarima

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.4 MB 2.3 MB/s 
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [448]:
#!c1.4
#%%
import pmdarima as pm

# Create model_folder
model_name =  "Auto ARIMA"
model_folder = os.path.join(exp_folder, model_name)
os.mkdir(model_folder)

for ticker_name in CONFIG['TARGET']['TICKERS']:
    target = targets_df[ticker_name].dropna()
    series_folder = os.path.join(model_folder, ticker_name.split('.')[0])
    os.mkdir(series_folder)

    cv_val_metrics, cv_test_metrics = [], []
    cv_val_predictions, cv_test_predictions = [], []

    for train_index, val_index in tscv.split(target):
        # divide val to val and test
        test_index = val_index[CONFIG["VAL_LEN"]:]
        val_index = val_index[:CONFIG["VAL_LEN"]]
        iter_folder = f"TRAIN_{len(train_index)}_VAL_{len(val_index)}_TEST_{len(test_index)}"
        print(iter_folder)

        # Create folder for cv iteration
        iter_folder = os.path.join(series_folder, iter_folder)
        plots_folder = os.path.join(iter_folder, 'plots')
        os.mkdir(iter_folder)
        os.mkdir(plots_folder)
        plot_id = 0

        target_train, target_val, target_test = target[train_index], target[val_index], target[test_index]
        seve_fig_from_array(target_train, plots_folder, f'{plot_id}_target')
        plot_id+=1

        arima_model = pm.auto_arima(target_train,
                                   start_p=1, start_q=1,
                                   test='adf',
                                   max_p=3, max_q=3, m=12,
                                   start_P=0, seasonal=True,
                                   d=None, D=1, trace=True,
                                   error_action='ignore',
                                   suppress_warnings=True,
                                   maxiter=10,
                                   n_jobs=-1,
                                   stepwise=True)
        predicted_arima = arima_model.predict(n_periods=CONFIG["TEST_LEN"] * 2)
        predicted_val = predicted_arima[:CONFIG["TEST_LEN"]]
        predicted_test = predicted_arima[CONFIG["TEST_LEN"]:]

        predicted_val = pd.Series(predicted_val, index=target_val.index)
        predicted_test = pd.Series(predicted_test, index=target_test.index)

        # Calculate metrics
        val_mertics = metrics_eval(target_val[:len(predicted_val)], predicted_val, save_dir=iter_folder, print_result=False, name="val_mertics")
        test_mertics = metrics_eval(target_test[:len(predicted_val)], predicted_test, save_dir=iter_folder, print_result=False, name='test_mertics')

        cv_val_metrics.append(val_mertics)
        cv_test_metrics.append(test_mertics)

        cv_val_predictions.append(predicted_val)
        cv_test_predictions.append(predicted_test)

        train_val_test_pred_plot(train=target_train,
                                 val=target_val,
                                 test=target_test,
                                 val_pred=predicted_val,
                                 test_pred=predicted_test,
                                 save_dir=plots_folder, show=False)
    models_dict[model_name] = {} if model_name not in models_dict else models_dict[model_name]
    models_dict[model_name]["mean_val_metrics"] = {} if "mean_val_metrics" not in models_dict[model_name] else models_dict[model_name]["mean_val_metrics"]
    models_dict[model_name]["mean_test_metrics"] = {} if "mean_test_metrics" not in models_dict[model_name] else models_dict[model_name]["mean_test_metrics"]
    models_dict[model_name]["mean_val_metrics"][ticker_name] = dict(pd.DataFrame(cv_val_metrics).mean())
    models_dict[model_name]["mean_test_metrics"][ticker_name] = dict(pd.DataFrame(cv_test_metrics).mean())

/home/jupyter/.local/lib/python3.7/site-packages/pmdarima/arima/_validation.py:77: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  'Falling back to stepwise parameter search.' % n_jobs)
/home/jupyter/.local/lib/python3.7/site-packages/pmdarima/arima/_validation.py:77: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  'Falling back to stepwise parameter search.' % n_jobs)
/home/jupyter/.local/lib/python3.7/site-packages/pmdarima/arima/_validation.py:77: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  'Falling back to stepwise parameter search.' % n_jobs)
/home/jupyter/.local/lib/python3.7/site-packages/pmdarima/arima/_validation.py:77: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  'Falling back to stepwise parameter search.' % n_jobs)
/hom

TRAIN_2698_VAL_7_TEST_7
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=4.49 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=34368.115, Time=0.15 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=28476.747, Time=3.60 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=32468.291, Time=4.44 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=34413.095, Time=0.10 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=29228.038, Time=0.57 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=inf, Time=9.19 sec
 ARIMA(1,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=4.51 sec
 ARIMA(1,0,0)(0,1,1)[12] intercept   : AIC=inf, Time=3.93 sec
 ARIMA(1,0,0)(2,1,1)[12] intercept   : AIC=inf, Time=9.05 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=34354.739, Time=2.73 sec
 ARIMA(2,0,0)(1,1,0)[12] intercept   : AIC=28532.192, Time=4.44 sec
 ARIMA(1,0,1)(1,1,0)[12] intercept   : AIC=28528.085, Time=4.36 sec
 ARIMA(0,0,1)(1,1,0)[12] intercept   : AIC=32447.960, Time=4.98 sec
 ARIMA(2,0,

In [450]:
#!c1.8
with open(os.path.join(exp_folder, "exp_result_3.json"), "w") as outfile:
    json.dump(models_dict, outfile, skipkeys=True)

for mode_name in ['val', 'test']:
    exp_metrics = (pd.DataFrame({model_key: pd.DataFrame(models_dict[model_key][f'mean_{mode_name}_metrics']).mean(axis=1).to_dict() for model_key in models_dict.keys()}).round(2)
                   .round(2)
                   .sort_values(by='Mean absolute percentage error', axis=1)
                   .transpose()
                   )
    exp_metrics.to_csv(os.path.join(exp_folder, f'exp_{mode_name}_metrics_3.csv'), mode='w', sep=';')
    display(exp_metrics)

,Mean Absolute Error,Mean Squared Error,Symmetric Mean absolute percentage error,Root Mean Squared Error,Mean absolute percentage error
None_None_Differencer_MinMaxScaler,40.42,9721.83,1.74,50.47,1.73
None_Deseasonalizer_Differencer_MinMaxScaler,36.13,6682.22,1.76,46.17,1.75
HampelFilter_None_Differencer_MinMaxScaler,32.63,3572.61,1.83,38.91,1.82
None_None_BoxCoxTransformer_MinMaxScaler,38.38,6367.11,1.90,49.38,1.90
None_None_None_MinMaxScaler,39.45,6961.98,1.96,49.26,1.95
HampelFilter_Deseasonalizer_Differencer_MinMaxScaler,38.75,5141.85,2.09,45.86,2.08
HampelFilter_None_None_MinMaxScaler,41.13,7249.00,2.11,50.21,2.10
HampelFilter_None_LogTransformer_MinMaxScaler,37.95,5466.08,2.14,46.69,2.12
None_None_Differencer_None,49.65,12343.92,2.12,60.11,2.13
None_Deseasonalizer_None_MinMaxScaler,41.01,6892.29,2.16,50.23,2.15


,Mean Absolute Error,Mean Squared Error,Symmetric Mean absolute percentage error,Root Mean Squared Error,Mean absolute percentage error
HampelFilter_None_Differencer_None,30.29,2845.95,1.82,38.00,1.82
None_None_Differencer_None,26.62,2264.68,1.86,33.04,1.87
HampelFilter_None_Differencer_MinMaxScaler,37.63,6003.46,2.17,45.67,2.14
None_None_Differencer_MinMaxScaler,35.27,4472.01,2.29,43.35,2.25
HampelFilter_Deseasonalizer_Differencer_None,33.47,3813.27,2.48,37.65,2.49
None_Deseasonalizer_Differencer_None,33.50,3897.84,2.50,37.86,2.50
HampelFilter_Deseasonalizer_Differencer_MinMaxScaler,45.24,7938.55,2.69,50.92,2.67
None_Deseasonalizer_Differencer_MinMaxScaler,45.18,7615.22,2.74,51.30,2.70
None_Deseasonalizer_None_MinMaxScaler,58.68,16055.82,2.77,63.28,2.75
None_None_None_MinMaxScaler,52.25,10588.28,2.79,57.47,2.77


In [449]:
#!c1.8
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:20]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                           _i1:  590.0 B
                    sizeof_fmt:  144.0 B
                           _ih:  104.0 B
                            In:  104.0 B
                   __builtin__:   88.0 B
                  __builtins__:   88.0 B
                           sys:   88.0 B
                           _dh:   80.0 B
                           Out:   80.0 B
                           _oh:   80.0 B
EphemeralVariable.ML_KERNEL_CWD:   77.0 B
                   get_ipython:   72.0 B
        execute_livy_statement:   72.0 B
                execute_script:   72.0 B
      del_datasphere_variables:   72.0 B
               HyperParameters:   64.0 B
                  mean_test_df:   64.0 B
                    targets_df:   64.0 B
                         model:   64.0 B
                  target_train:   64.0 B


/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:872: UserWarning: The following variables cannot be serialized: name
  warnings.warn(message)


In [ ]:
#!c1.8
